In [1]:
import numpy as np
import strawberryfields as sf
from strawberryfields import ops
import matplotlib.pyplot as plt
from scipy.linalg import block_diag

In [2]:
#Creating a 4 mode simulation for boson sampling
n_modes = 4 #Number of modes for the simulation (i.e., number of single photon sources)
BS = sf.Program(n_modes)

#The photonic quantum circuit
with BS.context as q:
    #q is a list of the modes
    #Initializing the input fock state: 2 input photons, Configuration - 1, 1, 0, 0
    ops.Fock(1) | q[0]
    ops.Fock(1) | q[1]
    ops.Fock(0) | q[2]
    ops.Fock(0) | q[3]
    #Rotations for mode quadratures
    ops.Rgate(0.5719)  | q[0]
    ops.Rgate(-1.9782) | q[1]
    ops.Rgate(2.0603)  | q[2]
    ops.Rgate(0.0644)  | q[3]
    #Beam splitter array for entanglement
    ops.BSgate(0.7804, 0.8578)  | (q[0], q[1])
    ops.BSgate(0.06406, 0.5165) | (q[2], q[3])
    ops.BSgate(0.473, 0.1176)   | (q[1], q[2])
    ops.BSgate(0.563, 0.1517)   | (q[0], q[1])
    ops.BSgate(0.1323, 0.9946)  | (q[2], q[3])
    ops.BSgate(0.311, 0.3231)   | (q[1], q[2])
    ops.BSgate(0.4348, 0.0798)  | (q[0], q[1])
    ops.BSgate(0.4368, 0.6157)  | (q[2], q[3])
    
#Initializing the simulation backend
engine = sf.Engine(backend="fock", backend_options={"cutoff_dim": 7})
#Running the simulation
results = engine.run(BS)
#Probabilities of measurements
probabilities = results.state.all_fock_probs()

In [3]:
#Certain state probabilities
print(f'The probability of measuring 0, 2, 0, 0 is {round(probabilities[0,2,0,0], 3)}')
print(f'The probability of measuring 0, 0, 1, 1 is {round(probabilities[0,0,1,1], 3)}')
print(f'The probability of measuring 1, 1, 0, 0 is {round(probabilities[1,1,0,0], 3)}')
print(f'The probability of measuring 1, 1, 1, 1 is {round(probabilities[1,1,1,1], 3)}')

The probability of measuring 0, 2, 0, 0 is 0.237
The probability of measuring 0, 0, 1, 1 is 0.034
The probability of measuring 1, 1, 0, 0 is 0.244
The probability of measuring 1, 1, 1, 1 is 0.0


In [4]:
#Unitary construction using strawberry fields functions

#Another 4 mode simulation program, this time for constructing the unitary
Unitary_prog = sf.Program(n_modes)

#Don't care about the initial fock states, that's why we look at the program from element 4
Unitary_prog.circuit = BS.circuit[4:]

#Get the unitary in the form of a gaussian symplectic matrix
Unitary_gaus_symplectic = Unitary_prog.compile(compiler="gaussian_unitary")
'''For more information on the gaussian symplectic matrix, look at: 
https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.GaussianTransform.html
'''

#Extracting the symplectic matrix from the gaussian symplectic matrix
Unitary_symplectic = Unitary_gaus_symplectic.circuit[0].op.p[0]

#Getting the unitary from the symplectic 
Unitary = Unitary_symplectic[:4, :4] + 1j*Unitary_symplectic[4:, :4]
print(np.round(Unitary, 4))

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.457 +0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.11  -0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


In [5]:
#Brute force way of calculating the Unitary, just so that I fully understand what the strawberry fields function does

#Unitary for the rotation gates
U_rot = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])

#The theta and phi arrays for the beam splitters
BS_thetas = np.array([0.7804, 0.06406, 0.473, 0.563, 0.1323, 0.311, 0.4348, 0.4368])
BS_phis = np.array([0.8578, 0.5165, 0.1176, 0.1517, 0.9946, 0.3231, 0.0798, 0.6157])

#The elements of the beam splitter unitary for each beam splitter in the circuit
Transmission_coeffs = np.cos(BS_thetas)
Refelction_coeffs = np.sin(BS_thetas)*np.exp(1j*BS_phis)

#A list of all the beam splitter unitaries
BS_unitaries = [np.array([[Transmission_coeffs[i], -1.*np.conj(Refelction_coeffs[i])], \
                          [Refelction_coeffs[i], Transmission_coeffs[i]]]) for i in range (len(Transmission_coeffs))]

#Making the 4x4 circuit moment unitaries - essentially unitaries of beam splitters carried out in parallel
BS1 = block_diag(*BS_unitaries[0:2])
BS2 = block_diag([[1]], BS_unitaries[2], [[1]])
BS3 = block_diag(*BS_unitaries[3:5])
BS4 = block_diag([[1]], BS_unitaries[5], [[1]])
BS5 = block_diag(*BS_unitaries[6:8])

#Final Unitary
Unitary_analytical = np.linalg.multi_dot([BS5, BS4, BS3, BS2, BS1, U_rot])
print(np.round(Unitary_analytical, 4))

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.457 +0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.11  -0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


In [6]:
#Confirming that both the unitaries are the same
print(np.round(Unitary_analytical, 4) == np.round(Unitary, 4))

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
